# CappingTransformer
This notebook shows the functionality in the CappingTransformer class. This transformer caps numeric columns at either a maximum value or minimum value or both. <br>

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing

In [2]:
import tubular
from tubular.capping import CappingTransformer

In [3]:
tubular.__version__

'0.3.0'

## Load California housing dataset from sklearn

In [4]:
cali = fetch_california_housing()
cali_df = pd.DataFrame(cali["data"], columns=cali["feature_names"])
# add nulls
cali_df["HouseAge"] = cali_df["HouseAge"].sample(frac=0.95, random_state=123)
cali_df["Population"] = cali_df["Population"].sample(frac=0.99, random_state=123)

In [5]:
cali_df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [6]:
cali_df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,20640.000000,19608.000000,20640.000000,20640.000000,20434.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.672532,5.429000,1.096675,1424.137075,3.070655,35.631861,-119.569704
std,1.899822,12.568207,2.474173,0.473911,1129.684881,10.386050,2.135952,2.003532
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000
25%,2.563400,18.000000,4.440716,1.006079,788.000000,2.429741,33.930000,-121.800000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000
75%,4.743250,37.000000,6.052381,1.099526,1722.750000,3.282261,37.710000,-118.010000
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000


## Simple usage
First we will demonstrate the case where the user has pre determined the values to cap at, and codes these directly into the transformer.

### Initialising CappingTransformer

The `CappingTransformer` should be initialised by specifying either a `capping_values` dict or a `quantiles` dict, but not both. <br>
These must be a `dict` where each key is a column to apply capping to and the items are lists of length 2, containing either `None` or numeric values;
- in the case of `capping_values` the user directly specifies the values to cap at giving a lower and upper value here 
- or if `quantiles` is specified the values in each list should be the quantiles to cap at (these will be determined when running the `fit` method) 
- if a `None` value is present then the relevant min or max capping is not applied. <br>

In the example below both min and max capping will be applied, if `0.66` was replaced with `None` then only min capping would be applied.

In [7]:
cap_1 = CappingTransformer(capping_values={"AveOccup": [1, 5.5]})

### CappingTransformer fit
In the case where the user specifies `capping_values` then there is no need to use the `fit` method. This is only required is the user specifies the `quantiles` argument when initialising the transformer. In fact if `capping_values` were specified and `fit` is called a warning will be generated.

### CappingTransformer transform
Note, if the transformer is applied to non-numeric columns then an exception will be raised.

In [8]:
cali_df["AveOccup"].quantile([0, 0.2, 0.8, 1.0])

0.0       0.692308
0.2       2.338822
0.8       3.424272
1.0    1243.333333
Name: AveOccup, dtype: float64

In [9]:
cali_df_2 = cap_1.transform(cali_df)

In [10]:
cali_df_2["AveOccup"].quantile([0, 1.0])

0.0    1.0
1.0    5.5
Name: AveOccup, dtype: float64

## Transform with nulls
Nulls are not imputed in the transform method. There are other transformers in the package to impute null values.

In [11]:
cap_2 = CappingTransformer({"HouseAge": [3, 50]})

In [12]:
cali_df["HouseAge"].isnull().sum()

1032

In [13]:
cali_df_3 = cap_2.transform(cali_df)

In [14]:
cali_df_3["HouseAge"].isnull().sum()

1032

## Transforming multiple columns
The specific capping values are applied to each column. The user has the option to set different combinations of min and max for each column.

In [15]:
cap_3 = CappingTransformer(
    capping_values={"AveOccup": [None, 10], "HouseAge": [None, 50]}
)

In [16]:
cali_df[["AveOccup", "HouseAge"]].max()

AveOccup    1243.333333
HouseAge      52.000000
dtype: float64

In [17]:
cali_df_4 = cap_3.transform(cali_df)

In [18]:
cali_df_4[["AveOccup", "HouseAge"]].max()

AveOccup    10.0
HouseAge    50.0
dtype: float64

## Capping at quantiles
If the user does not want to pre specify quantiles up front but rather set them at quantiles in the data then the user can use the `quantiles` argument instead of `capping_values` when initialising the transformer. <br>
This takes the same structure as `capping_values` but the user is specifying quantiles to cap at instead of the values directly. <br>
The `fit` method of the transformer calculates the requested quantiles for each column. If the user also specified `weights_column` when initialising the transformer then weighted quantiles will be calculated using that column in `X`. <br>
The user must run `fit` before running `transform` when using quantiles, otherwise and exception will be raised. 

In [19]:
cap_4 = CappingTransformer(
    quantiles={"HouseAge": [None, 0.8], "AveRooms": [0.1, None], "AveOccup": [0.2, 0.8]}
)

In [20]:
cap_4.fit(cali_df)

CappingTransformer(capping_values={'AveOccup': [2.338815789473684,
                                                3.4242424242424243],
                                   'AveRooms': [3.7904191616766467, None],
                                   'HouseAge': [None, 40.0]},
                   quantiles={'AveOccup': [0.2, 0.8], 'AveRooms': [0.1, None],
                              'HouseAge': [None, 0.8]})

In [21]:
cali_df[["HouseAge", "AveRooms", "AveOccup"]].min()

HouseAge    1.000000
AveRooms    0.846154
AveOccup    0.692308
dtype: float64

In [22]:
cali_df[["HouseAge", "AveRooms", "AveOccup"]].max()

HouseAge      52.000000
AveRooms     141.909091
AveOccup    1243.333333
dtype: float64

In [23]:
cali_df_5 = cap_4.transform(cali_df)

In [24]:
cali_df_5[["HouseAge", "AveRooms", "AveOccup"]].min()

HouseAge    1.000000
AveRooms    3.790419
AveOccup    2.338816
dtype: float64

In [25]:
cali_df_5[["HouseAge", "AveRooms", "AveOccup"]].max()

HouseAge     40.000000
AveRooms    141.909091
AveOccup      3.424242
dtype: float64

## Weighted quantiles
The user can also specify the `weights_column` argument when initialising the transformer, so that `fit` will use the column in the input data `X` when calculating the quantiles.

### Equal weights
First we will use an equal weight column and demonstrate we can recover the same quantiles from `cap_4` where no `weights_column` was specified.

In [26]:
cali_df["weights"] = 1

In [27]:
cap_5 = CappingTransformer(quantiles={"AveOccup": [0.2, 0.8]}, weights_column="weights")

In [28]:
cap_5.fit(cali_df)

CappingTransformer(capping_values={'AveOccup': [2.338815789473684,
                                                3.4242424242424243]},
                   quantiles={'AveOccup': [0.2, 0.8]},
                   weights_column='weights')

In [29]:
cap_5.capping_values["AveOccup"] == cap_4.capping_values["AveOccup"]

True

### Non-equal weights
Next we will set weights to zero for values that are beyond the `[0.2, 0.8]` quantiles identified previously, and check that the `[0, 1]` weighted quantiles identified are the min and max, where the weight is > 0.

In [30]:
cali_df["weights2"] = 1
cali_df.loc[cali_df["AveOccup"] < 2.338815789473684, "weights2"] = 0
cali_df.loc[cali_df["AveOccup"] > 3.4242424242424243, "weights2"] = 0

In [31]:
cap_6 = CappingTransformer(
    quantiles={"AveOccup": [0.0, 1.0]}, weights_column="weights2"
)

In [32]:
cap_6.fit(cali_df)

CappingTransformer(capping_values={'AveOccup': [2.338815789473684,
                                                3.4242424242424243]},
                   quantiles={'AveOccup': [0.0, 1.0]},
                   weights_column='weights2')

In [33]:
cap_6.capping_values["AveOccup"]

[2.338815789473684, 3.4242424242424243]

In [34]:
cali_df.loc[cali_df["weights2"] > 0, "AveOccup"].min(), cali_df.loc[
    cali_df["weights2"] > 0, "AveOccup"
].max()

(2.338815789473684, 3.4242424242424243)

### Weights columns restrictions 
The weights column can have 0 values in it, but it cannot have `null` values, `np.Inf`, `-np.Inf` or negative values.

In [35]:
cap_7 = CappingTransformer(
    quantiles={"AveOccup": [0.1, 0.8]}, weights_column="Population"
)

In [36]:
cali_df["Population"].fillna(0, inplace=True)

In [37]:
cap_7.fit(cali_df)

CappingTransformer(capping_values={'AveOccup': [2.1656277821907675,
                                                3.556821778094737]},
                   quantiles={'AveOccup': [0.1, 0.8]},
                   weights_column='Population')

In [38]:
cap_7.capping_values["AveOccup"]

[2.1656277821907675, 3.556821778094737]